**1 (a) Let $\theta \in \mathbb{R}^d$ be a vector and $\theta_0 \in \mathbb{R}$ be a constant. Let $x = [x_1, ..., x_d]$ be a vector of unknowns. Consider the hyperplane in $\mathbb{R}^d$ whose equation is given by $\theta\cdot x + \theta_0 = 0$. Given a point $y \in \mathbb{R}^d$, find its distance to the hyperplane.** 

![alt text](https://github.com/charleswongzx/machine-learning-01.112/blob/master/HW1-1.jpg?raw=true)

**1 (b) Let $X$ and $Y$ be independent Poisson random variables, i.e. **
\begin{equation}
  \mathbb{P}(X = x) = \frac{\alpha^x e^{-\alpha}}{x!}, \:\:\: \mathbb{P}(Y = y) = \frac{\beta^y e^{-\beta}}{y!}, \:\:\: \text{for all } x,y \geq 0
\end{equation}
**for some rates $\alpha, \beta > 0$. Show that $Z = X + Y$ is also Poisson, and find its rate $\gamma$. **

![alt text](https://github.com/charleswongzx/machine-learning-01.112/blob/master/HW1-2.jpg?raw=true)

***2 (a) Write down the version of Python and the version of Theano you are using.***

In [4]:
import sys
import theano
print ('Python version:', sys.version)
print('Theano version:', theano.__version__)

Python version: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
Theano version: 1.0.3


***3 (a) Write down the vector w of coefficients computed by gradient descent.***

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time

csv = 'https://www.dropbox.com/s/oqoyy9p849ewzt2/linear.csv?dl=1'
data = np.genfromtxt(csv,delimiter=',')
X = data[:,1:]
Y = data[:,0]

In [0]:
import theano.tensor as T
d = X.shape[1] # dimension of feature vectors
n = X.shape[0] # number of training samples
learn_rate = 0.5 # learning rate for gradient descent

In [0]:
x = T.matrix(name='x') # feature matrix
y = T.vector(name='y') # response vector
w = theano.shared(np.zeros((d,1)),name='w') # model parameters

In [0]:
risk = T.sum((T.dot(x,w).T - y)**2)/2/n # empirical risk
grad_risk = T.grad(risk, wrt=w) # gradient of the risk

In [0]:
train_model = theano.function(inputs=[],
outputs=risk,
updates=[(w, w-learn_rate*grad_risk)],
givens={x:X, y:Y})

In [10]:
n_steps = 50
gd_start = time.time()

for i in range(n_steps):
  print('Risk:',train_model())
  print('w=',w.get_value())
  
gd_end = time.time()
print("Gradient descent time elapsed:", gd_end-gd_start, "seconds")

Risk: 2.619322008585456
w= [[-0.40989365]
 [ 0.58200428]
 [-0.03422747]
 [-0.84429332]]
Risk: 0.7587559422725657
w= [[-0.56425042]
 [ 0.91043645]
 [-0.03598075]
 [-1.30070194]]
Risk: 0.2354281222497302
w= [[-0.61197745]
 [ 1.09756651]
 [-0.02729178]
 [-1.55159757]]
Risk: 0.07939576047330815
w= [[-0.61854675]
 [ 1.2052004 ]
 [-0.0168923 ]
 [-1.69176458]]
Risk: 0.0300968963645766
w= [[-0.61159653]
 [ 1.26766213]
 [-0.00782182]
 [-1.77127007]]
Risk: 0.013695201975103273
w= [[-6.02022776e-01]
 [ 1.30420333e+00]
 [-7.98156865e-04]
 [-1.81700086e+00]]
Risk: 0.0080060923088713
w= [[-0.59358047]
 [ 1.32573352]
 [ 0.00431825]
 [-1.84363622]]
Risk: 0.005970446508700007
w= [[-0.58715742]
 [ 1.33849761]
 [ 0.00791253]
 [-1.85932124]]
Risk: 0.005225932117259178
w= [[-0.58261402]
 [ 1.34610447]
 [ 0.01037892]
 [-1.86864581]]
Risk: 0.004949550127903777
w= [[-0.57953393]
 [ 1.35065778]
 [ 0.01204429]
 [-1.8742339 ]]
Risk: 0.004845924776165539
w= [[-0.57750185]
 [ 1.35339322]
 [ 0.0131559 ]
 [-1.877605

***b) Write a program which calculates the exact solution for the optimal coefficients in the linear regression problem. Compare your solution with the answer in (a).***

In [11]:
from numpy.linalg import inv

la_start = time.time()
b = inv(X.T.dot(X)).dot(X.T).dot(Y)  # matrix linear regression inverse
la_end = time.time()

print('Linear inv time elapsed:', la_end-la_start, 'seconds')

Linear inv time elapsed: 0.005253791809082031 seconds


In [12]:
print(b.shape)

(4,)


The result calculated using simple linear algebra matches the result from gradient descent. It is noted that the method in (b) was an order of magnitude faster at 0.0130s compared to (a) at 0.535s. This is, however, influenced by n_steps declared for (a). (a) appears to converge on the solution before step 50, and can be optimised further.

A possible optimisation is to end subsequent descent steps after matching parameters between step n and n-1 have been achieved.

***(c) Find a Python library that computes the optimal coefficients in the linear regression
problem, and compare its solution with the answer in (a).***

In [13]:
from sklearn.linear_model import LinearRegression

lm_start = time.time()

lm = LinearRegression(fit_intercept=True)
lm.fit(X,Y)

lm_end = time.time()

print(lm.coef_, '\n', lm.intercept_)
print('SKlearn time elapsed:', lm_end - lm_start, 'seconds')

[-0.57392068  1.35757059  0.01527565  0.        ] 
 -1.8828807643601515
SKlearn time elapsed: 0.0012409687042236328 seconds


SKlearn's LinearRegression() method is far quicker than previous solutions, and converges on a matching answer.

***(d) Bonus (1 point). Write a program which computes the solution using stochastic
gradient descent. You may use a minibatch size of 5 data points. For convergence,
remember to decrease the learning rate over time.***

In [0]:
index = T.lscalar(name='index')  # integer scalar for running through minibatch
batch_size = 5

sgd_X = theano.shared(X, name='sgd_X')
sgd_Y = theano.shared(Y, name='sgd_Y')

sgd_x = T.matrix(name='sgd_x') # feature matrix
sgd_y = T.vector(name='sgd_y') # response vector
sgd_w = theano.shared(np.zeros((d,1)),name='sgd_w') # model parameters
learn_rate = theano.shared(0.7, name='learn_rate')

sgd_risk = T.sum((T.dot(sgd_x,sgd_w).T - sgd_y)**2)/2/n # empirical risk
sgd_grad_risk = T.grad(sgd_risk, wrt=sgd_w) # gradient of the risk


In [15]:
converged = False
coeffs = np.array([0,0,0,0])

train_model = theano.function(inputs=[index],
                              outputs=sgd_risk,
                              updates=[(sgd_w, sgd_w-learn_rate*sgd_grad_risk), 
                                       (learn_rate, learn_rate-0.05)],
                              givens={sgd_x: sgd_X[index * batch_size: (index+1) * batch_size], 
                                      sgd_y: sgd_Y[index * batch_size: (index+1) * batch_size]})

# train_model = theano.function(inputs=[],
#                               outputs=sgd_risk,
#                               updates=[(sgd_w, sgd_w-learn_rate*sgd_grad_risk), 
#                                        (learn_rate, learn_rate-0.02)],
#                               givens={sgd_x:X, 
#                                       sgd_y:Y})
sgd_start = time.time()
index = 0

while converged is False:
  
  print(train_model(index))
#   print(train_model())
  print(sgd_w.get_value())
  
  # Exit loop if coefficients converge to a close enough value
  if np.allclose(coeffs, sgd_w.get_value(), rtol=1e-07, atol=1e-10):
    sgd_end = time.time()
    print('Result converged')
    converged = True
    
  coeffs = sgd_w.get_value()
  
  index += 1
  if index > 10:
    index = 0
  
print('SGD elapsed time:', sgd_end - sgd_start, 'seconds')

print(sgd_w.shape)

0.11992797019774265
[[ 0.05223588]
 [ 0.00644617]
 [ 0.00348568]
 [-0.10106731]]
0.2747017234205148
[[ 0.07019628]
 [ 0.12947361]
 [-0.12628328]
 [-0.22009629]]
0.3443017464721848
[[-0.01968061]
 [ 0.24333349]
 [-0.07641392]
 [-0.33718307]]
0.15672558694148764
[[-0.05948728]
 [ 0.25723598]
 [-0.03317387]
 [-0.4204885 ]]
0.1601479128034408
[[-0.08209255]
 [ 0.32422439]
 [-0.03634549]
 [-0.4703045 ]]
0.2715426229463849
[[-0.22281921]
 [ 0.39518957]
 [ 0.02364032]
 [-0.54233931]]
0.12069256599139921
[[-0.25227445]
 [ 0.42422023]
 [-0.00411661]
 [-0.58725968]]
0.1181507058659389
[[-0.27509496]
 [ 0.44453467]
 [-0.01501336]
 [-0.63242975]]
0.15739194220561442
[[-0.29052785]
 [ 0.48052456]
 [-0.01434765]
 [-0.67591346]]
0.019079597249791146
[[-0.27927939]
 [ 0.4807831 ]
 [-0.01999239]
 [-0.68580205]]
0.0
[[-0.27927939]
 [ 0.4807831 ]
 [-0.01999239]
 [-0.68580205]]
Result converged
SGD elapsed time: 0.011999845504760742 seconds
Shape.0
